In [2]:
from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt

from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
import keras
from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
import warnings

warnings.filterwarnings('ignore')

In [3]:
# Function to generate explanations
# Function explanation for lime

def get_lime_explanation(data, e, L2X=False) :
    # e  = fn = model.predict
    # Iniatialisation
    df_expplanation = pd.DataFrame(columns=[str(i) for i in range(data.shape[1]*data.shape[2])])

    # Get explanations
    for row in range(data.shape[0]) : 
        explanation = lime_explainer.explain_instance(data[row],
                                                      e,
                                                      num_features=data.shape[1]*data.shape[2]) 
        # fn = model.predict, initialize lime_explainer = Reccurent()
        lime_values = explanation.local_exp[1]
        # Add explanation in df_explanation
        lime_dict = {}
        for tup in lime_values :
            lime_dict[str(tup[0])] = tup[1]
        df_expplanation.loc[len(df_expplanation)] = lime_dict
    
    return df_expplanation

# # Function explanation for others
def get_explainations(data, e, L2X = False) :
    
    # df diemnsion
    if L2X==True :
        X_to_def_col = data[0:1]
        explanation_test = e.explain(X_to_def_col.reshape((X_to_def_col.shape[0], -1)))
        num_columns = explanation_test.flatten().shape[0]
        
    else : 
        explanation_test = e.explain(data[0:1])
        num_columns = explanation_test.flatten().shape[0]
    
    # Iniatialisation
    df_expplanation = pd.DataFrame(columns=[str(i) for i in range(num_columns)])

    # Get explanations
    for row in range(data.shape[0]) :
        if L2X==True:
            X_row = data[row:row+1]
            explanation = e.explain(X_row.reshape((X_row.shape[0], -1)))
        else :
            explanation = e.explain(data[row:row+1])
        # Add explanation in df_explanation
        explanation = explanation.flatten()
        feature_dict = {}
        for i in range(num_columns) :
            feature_dict[str(i)] = explanation[i]
        df_expplanation.loc[len(df_expplanation)] = feature_dict
    
    return df_expplanation

In [4]:
# Data preparation
# Data loading
train, test, y_test = prepare_data('FD003.txt')
print(train.shape, test.shape, y_test.shape)
train['RUL'].clip(upper=125, inplace=True)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors] 

(24720, 27) (16596, 26) (100, 1)


In [5]:
models = {
    'model_1l' : model003,
    # 'model_2l' : create_model2C003(),
    # 'model_3l' : create_model3C003(),
    # 'model_4l' : create_model4C003(),
    }

In [6]:
from tqdm import tqdm
for elm in tqdm(models.keys()):
    if elm == 'model_1l':
        print(elm)

100%|██████████| 1/1 [00:00<00:00, 4718.00it/s]

model_1l


In [6]:
# Modeling
# Fixe best hyperparameter
train['RUL'].clip(upper=125, inplace=True)
upper = 125
sequence_length = 40
alpha = 0.2
epochs = 20
batch_size = 64

mse = []
list_d = []
results = pd.DataFrame()
df_metrics = pd.DataFrame()

# Data prepration
X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

# create sequences train, test
train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
                for unit_nr in X_test_interim['Unit'].unique())
        
test_array = np.concatenate(list(test_gen)).astype(np.float32)



for model_name in tqdm(models.keys()):

    weights_file = 'lstm003_1C.h5'
    input_shape = (sequence_length, len(remaining_sensors))
    if model_name == 'model_1l':
        model = model = model003(input_shape, weights_file)
    else : 
        model = models[model_name]

    # test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,upper)
    print(train_array.shape, label_array.shape, test_array.shape)
    if True : 
        mse_val = []
        R2_val = []
        RMSE = []
        score_val = []
        gss = GroupShuffleSplit(n_splits=3, train_size=0.80, random_state=42) 
        test_rul = rul_piecewise_fct(y_test,upper)
        if model_name == 'model_1l':
            with tf.device('/device:GPU:0'):
                for train_unit, val_unit in gss.split(X_train_interim['Unit'].unique(), groups=X_train_interim['Unit'].unique()):
                    train_unit = X_train_interim['Unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
                    train_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, train_unit)
                    train_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], train_unit)

                    val_unit = X_train_interim['Unit'].unique()[val_unit]
                    val_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, val_unit)
                    val_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], val_unit)

                    # train and evaluate model
                    # model.compile(loss='mse', optimizer='adam')
                    model.load_weights(weights_file)  # reset optimizer and node weights before every training iteration

                    # with tf.device('/device:GPU:0'):
                    history = model.fit(train_split_array, train_split_label,
                                        validation_data=(val_split_array, val_split_label),
                                        epochs=epochs,
                                        batch_size=batch_size,
                                        verbose=0)
        #             mse.append(history.history['val_loss'][-1])
                    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
                                    for unit_nr in X_test_interim['Unit'].unique())
                    test_array = np.concatenate(list(test_gen)).astype(np.float32)
                    mse_val.append(history.history['val_loss'][-1])
                    y_hat_val_split = model.predict(test_array)
                    R2_val.append(r2_score(test_rul, y_hat_val_split))
                    RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
                    score_val.append(compute_s_score(test_rul, y_hat_val_split))
        
        else : 
            with tf.device('/device:GPU:0'):
                history = model.fit(train_array, label_array,
                                        validation_data=(test_array, test_rul),
                                        epochs=epochs,
                                        batch_size=batch_size,
                                        verbose=0)
                # mse.append(history.history['val_loss'][-1])
                mse_val.append(history.history['val_loss'][-1])
                y_hat_val_split = model.predict(test_array)
                R2_val.append(r2_score(test_rul, y_hat_val_split))
                RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
                score_val.append(compute_s_score(test_rul, y_hat_val_split))
                    
        d = {'RMSE_val':np.sqrt(mse_val),'RMSE_test': RMSE,'R2_test':np.mean(R2_val), 'Score_test':np.mean(score_val),
                    'alpha':alpha, 'rul_piecewise':upper, 'sequence_length':sequence_length, 'model' : model_name}
        list_d.append(d)
        results = pd.concat([results, pd.DataFrame(d)], ignore_index=True)
        results.to_csv('results/result003/performance_by_layerl.csv')

        for rd in range(5):

            # Get explanation and calculate the score
            # Echantillonage
            n_individus = test_array.shape[0]

            # # Choisir aléatoirement 5 indices d'individus
            np.random.seed(rd)
            indices_choisis = np.random.choice(n_individus, size=5, replace=False)

            # Sélectionner les données correspondant aux indices choisis
            test_array_sampling = test_array[indices_choisis, :, :]
            label_array_sampling = y_test.values[indices_choisis, :]

            # Afficher les dimensions des données sélectionnées
            print(test_array_sampling.shape, label_array_sampling.shape)
            
            # distance matrix XX'
            X_dist = pd.DataFrame(squareform(pdist(test_array_sampling.reshape((test_array_sampling.shape[0], -1)))))

            # LIME
            lime_explainer = RecurrentTabularExplainer(test_array, training_labels=label_array,
                                                            feature_names=remaining_sensors,
                                                            mode = 'regression',
                                                            )
            lime_values = get_lime_explanation(test_array_sampling, e = model.predict)
            lime_values.shape
            Lime_dist = pd.DataFrame(squareform(pdist(lime_values))) # Lime values explanation matrix

            #Lime's metrics
            list_metrics_lime = {}
            list_metrics_lime['identity'] = identity(X_dist, Lime_dist)
            list_metrics_lime['separability'] = separability(X_dist, Lime_dist)
            list_metrics_lime['stability'] = stability(X_dist, Lime_dist)
            list_metrics_lime['coherence'], list_metrics_lime['completness'], list_metrics_lime['congruence'] = coherence(model=model.predict, 
                                                            explainer = get_lime_explanation,
                                                        samples=test_array_sampling,
                                                            targets=label_array, e = model.predict)
            list_metrics_lime['selectivity'] = selectivity(model=model.predict, explainer = get_lime_explanation,
                                                        samples=test_array_sampling, e_x = model.predict)
            list_metrics_lime['accumen'] = acumen(get_lime_explanation, test_array_sampling, e=model.predict)
            list_metrics_lime['Verm_stability'] = stability_Velmurugan(get_lime_explanation, test_array_sampling,
                                                                        e=model.predict, top_features=200)
            list_metrics_lime['fidelity'], list_metrics_lime['sparsity'] = fidelity(model=model.predict, 
                                                            explainer = get_lime_explanation,
                                                            samples=test_array_sampling,
                                                            e = model.predict, L2X=True)
            list_metrics_lime['instability'] = instability(model=model.predict, 
                                                            explainer = get_lime_explanation,
                                                            samples=test_array_sampling,
                                                            e = model.predict, L2X=True)
            
            list_metrics_lime['alpha'] = alpha
            list_metrics_lime['upper'] = upper
            list_metrics_lime['sequence_length'] = sequence_length
            list_metrics_lime['explainer'] = 'lime'
            list_metrics_lime['model'] = model_name

            df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_lime])])

            # SHAP
            e = KernelSHAP(model)
            shapvalues = get_explainations(test_array_sampling, e)
            shapvalues.shape

            list_metrics_shap = {}
            shap_dist = pd.DataFrame(squareform(pdist(shapvalues))) # shap values explanation matrix

            list_metrics_shap['identity'] = identity(X_dist, shap_dist)
            list_metrics_shap['separability'] = separability(X_dist, shap_dist)
            list_metrics_shap['stability'] = stability(X_dist, shap_dist)
            list_metrics_shap['coherence'], list_metrics_shap['completness'], list_metrics_shap['congruence'] = coherence(model=model.predict, 
                                                            explainer = get_explainations,
                                                        samples=test_array_sampling,
                                                            targets=label_array, e = e)
            list_metrics_shap['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                            samples=test_array_sampling, e_x=e)
            list_metrics_shap['accumen'] = acumen(get_explainations, test_array_sampling, e=e)
            list_metrics_shap['Verm_stability'] = stability_Velmurugan(get_explainations, test_array_sampling,
                                                                        e=e, top_features=200)
            list_metrics_shap['fidelity'], list_metrics_shap['sparsity']= fidelity(model=model.predict, 
                                                            explainer = get_explainations,
                                                            samples=test_array_sampling,
                                                            e = e)
            list_metrics_shap['instability']= instability(model=model.predict, 
                                                            explainer = get_explainations,
                                                            samples=test_array_sampling,
                                                            e = e)
            
            list_metrics_shap['alpha'] = alpha
            list_metrics_shap['upper'] = upper
            list_metrics_shap['sequence_length'] = sequence_length
            list_metrics_shap['explainer'] = 'shap'
            list_metrics_shap['model'] = model_name

            df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_shap])])
            
            # L2X
            e = L2X(model.predict, test_array_sampling)
            l2xvalues = get_explainations(test_array_sampling, e, L2X=True)
            l2xvalues.shape

            # L2X's metrics
            list_metrics_l2x = {}
            l2x_dist = pd.DataFrame(squareform(pdist(l2xvalues))) # Lime values explanation matrix

            list_metrics_l2x['identity'] = identity(X_dist, l2x_dist)
            list_metrics_l2x['separability'] = separability(X_dist, l2x_dist)
            list_metrics_l2x['stability'] = stability(X_dist, l2x_dist)
            list_metrics_l2x['coherence'], list_metrics_l2x['completness'], list_metrics_l2x['congruence'] = coherence(model=model.predict, explainer = get_explainations,
                                                        samples=test_array_sampling, targets=label_array_sampling, e = e, L2X=True)
            list_metrics_l2x['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                            samples=test_array_sampling, e_x=e, L2X=True)
            list_metrics_l2x['accumen'] = acumen(get_explainations, test_array_sampling, e=e, L2X=True)
            list_metrics_l2x['Verm_stability'] = stability_Velmurugan(get_explainations, test_array_sampling,
                                                                        e=e, top_features=200, L2X=True)
            list_metrics_l2x['fidelity'], list_metrics_l2x['sparsity']= fidelity(model=model.predict, 
                                                            explainer = get_explainations,
                                                            samples=test_array_sampling,
                                                            e = e, L2X=True)
            list_metrics_l2x['instability'] = instability(model=model.predict, 
                                                            explainer = get_explainations,
                                                            samples=test_array_sampling,
                                                            e = e, L2X=True)
            
            list_metrics_l2x['alpha'] = alpha
            list_metrics_l2x['upper'] = upper
            list_metrics_l2x['sequence_length'] = sequence_length
            list_metrics_l2x['explainer'] = 'l2x'
            list_metrics_l2x['model'] = model_name


            df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_l2x])])
        df_metrics.to_csv('results/result003/quality_by_layerl.csv')

  0%|          | 0/4 [00:00<?, ?it/s]WARNING:absl:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.


(20820, 40, 14) (20820, 1) (100, 40, 14)
4/4 [==============================] - 0s 2ms/step
(5, 40, 14) (5, 1)
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==============================] - 0s 2ms/step
10
157/157 [==========

1/1 [==============================] - ETA: 0s - loss: 7256.5601 - mse: 7255.7954WARNING:tensorflow:5 out of the last 3964 calls to <function Model.make_test_function.<locals>.test_function at 0x31a97a830> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


1/1 [==============================] - 1s 539ms/step - loss: 7256.5601 - mse: 7255.7954 - val_loss: 7245.6807 - val_mse: 7244.9243


1/1 [==============================] - ETA: 0s - loss: 7254.6890 - mse: 7253.9243WARNING:tensorflow:6 out of the last 3965 calls to <function Model.make_test_function.<locals>.test_function at 0x325d7ff40> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


...

 50%|█████     | 2/4 [3:29:50<3:31:46, 6353.47s/it]

(20820, 40, 14) (20820, 1) (100, 40, 14)
4/4 [==============================] - 1s 6ms/step
(5, 40, 14) (5, 1)
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 6ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==========

 75%|███████▌  | 3/4 [5:36:28<1:55:21, 6921.53s/it]

(20820, 40, 14) (20820, 1) (100, 40, 14)
4/4 [==============================] - 1s 7ms/step
(5, 40, 14) (5, 1)
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==========

100%|██████████| 4/4 [8:17:47<00:00, 7466.80s/it]  


In [7]:
results

,RMSE_val,RMSE_test,R2_test,Score_test,alpha,rul_piecewise,sequence_length,model
0,41.821500,41.052021,-0.089196,27071.044085,0.2,125,40,model_1l
1,41.690219,40.781787,-0.089196,27071.044085,0.2,125,40,model_1l
2,41.622465,40.792150,-0.089196,27071.044085,0.2,125,40,model_1l
3,13.342692,13.342693,0.883944,409.152351,0.2,125,40,model_2l
4,12.744678,12.744678,0.894114,298.201819,0.2,125,40,model_3l
5,13.510454,13.510455,0.881007,407.046317,0.2,125,40,model_4l
